In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import time
from typing import Dict, List
import pickle

In [2]:
def url_to_soup(url: str, headers: Dict) -> BeautifulSoup:
  url_request = urllib.request.Request(url, data=None, headers=headers)
  url_response = urllib.request.urlopen(url_request)
  soup = BeautifulSoup(url_response, "html.parser", from_encoding="iso-8859-1")

  return soup

In [3]:
headers = {
           'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
           'cookie': 'visid_incap_1712983=Dfo+CVLdTCSzMHcIXEgPSb1y7l0AAAAAQUIPAAAAAAAetg/EHDkFJUigbIo4eaK4; incap_ses_532_1712983=dFrgDpkdqkYCF1u+mQxiB71y7l0AAAAAksMc42V5CJx6OdUZdeHflA==; has_js=1; _ga=GA1.2.626207180.1575908117; _gid=GA1.2.512657448.1575908117; __utma=158387685.626207180.1575908117.1575908117.1575908117.1; __utmc=158387685; __utmz=158387685.1575908117.1.1.utmcsr=(direct)^|utmccn=(direct)^|utmcmd=(none); __utmt=1; __utmt_b=1; _aeaid=5e12b9d6-0171-4fde-8ccf-1bba809a1bb2; aeatstartmessage=true; __utmb=158387685.4.10.1575908117',
          }


url = "https://en.wikipedia.org/wiki/List_of_current_NBA_team_rosters"
soup = url_to_soup(url, headers)

In [4]:
tables = soup.find_all("table", {"class": "toccolours"})

In [5]:
table = tables[0]
team_name = table.find("div", {"class": "navbar-header"}).b.text

team_name

'Boston Celtics roster'

In [6]:
d = dict()
for table in tables:
  # team_name
  team_name = table.find("div", {"class": "navbar-header"}).b.text

  uls = table.find_all("ul")

  # head_coach
  head_coach = uls[1].find("li").text

  # assistant_coaches
  assistant_coaches = []
  for li in uls[2]:
    try:
      assistant_coaches.append(li.text)
    except:
      continue

  # players
  players = []

  player_table = table.find("tbody").find("tbody")
  for tr in player_table.find_all("tr"):
    temp = []
    for td in tr.find_all("td"):
      temp.append(td.text)

    if temp != []:
      players.append(temp)

  d[team_name] = {
      "head_coach": head_coach,
      "assistant_coaches": assistant_coaches,
      "players": pd.DataFrame(players, columns=["position", "number", "name", "height", "weight", "dob", "from"])
  }

In [7]:
with open("nba_teams.pickle", "wb") as f:
  pickle.dump(d, f, protocol=pickle.HIGHEST_PROTOCOL)